[Dane z tej strony](https://huggingface.co/datasets/vargr/private_instagram/tree/refs%2Fconvert%2Fparquet/default/train)

### Zadanie 1
Wczytaj pliki danych i scal je w jedną ramkę DataFrame. Wykonaj analizę typów danych podobnie jak w przykładach. Zmierz wielkość pamięci RAM ramki z domyślnymi typami danych.

In [1]:
import fastparquet
import pandas as pd
import numpy

df1 = pd.read_parquet('0000.parquet', engine='fastparquet')
df2 = pd.read_parquet('0001.parquet', engine='fastparquet')

df = pd.concat(df1, df2)

display(df.head())
df.info()

FileNotFoundError: [Errno 2] No such file or directory: '0000.parquet'

In [ ]:
df.memory_usage()

In [ ]:
sum(df.memory_usage())

In [ ]:
sum(df.memory_usage(deep=True))

In [ ]:
def sizeof_fmt(num, suffix="B"):
    for unit in ("", "Ki", "Mi", "Gi", "Ti", "Pi", "Ei", "Zi"):
        if abs(num) < 1024.0:
            return f"{num:3.1f}{unit}{suffix}"
        num /= 1024.0
    return f"{num:.1f}Yi{suffix}"

In [ ]:
sizeof_fmt(sum(df.memory_usage(deep=True)))

### Zadanie 2
Dobierz bardziej optymalne typy danych i ponownie zmierz wielkość zajmowanej pamięci RAM. Porównaj obie wielkości na wykresie (wybierz pasujący typ wykresu).

In [ ]:
from datetime import datetime

start = datetime.now()
# new_df = pd.read_csv('zamowienia_expanded.csv', header=0)
print(f"Czas wczytywania case 1: {datetime.now() - start} sekund")

In [ ]:
def count_time(func):
    def wrapper(*args, **kwargs):
        start = datetime.now()
        func(*args, **kwargs)
        print(f"Czas wczytywania {func.__name__}: {datetime.now() - start} sekund")
        return func(*args, **kwargs)
    return wrapper

In [ ]:
df1.describe()

In [ ]:
pd.options.display.float_format = '{:.5f}'.format
df1.describe()

In [ ]:
for column in df1.columns:
    print(f'{column}: {sizeof_fmt(df1[column].memory_usage(deep=True))}')

In [ ]:
sizeof_fmt(df1['idZamowienia'].astype(np.int16).memory_usage(deep=True))

In [ ]:
sizeof_fmt(df1['Kraj'].astype('category').memory_usage(deep=True))

In [ ]:
sizeof_fmt(df1['Sprzedawca'].astype('category').memory_usage(deep=True))

In [ ]:
sizeof_fmt(pd.to_datetime(df1['Data zamowienia']).memory_usage(deep=True))

In [ ]:
# tworzymy pustą ramkę danych, aby przechować w niej dane w nowym, bardziej optymalnym formacie
df2 = pd.DataFrame()

In [ ]:
# zmieniamy format niektórych kolumn
df2['Kraj'] = df1['Kraj'].astype('category')
df2['Sprzedawca'] = df1['Sprzedawca'].astype('category')
df2['Data zamowienia'] = pd.to_datetime(df1['Data zamowienia'])
df2['idZamowienia'] = df1['idZamowienia'].astype(np.int16)
df2['Utarg'] = df1['Utarg']

In [ ]:
sizeof_fmt(sum(df2.memory_usage(deep=True)))

In [ ]:
# możemy również spróbować wykonać downcasting dla kolumn numerycznych wykorzystując wbudowaną funkcję biblioteki panda to_numeric
utarg_downcast = pd.to_numeric(df2["Utarg"], downcast='float')
sizeof_fmt(utarg_downcast.memory_usage(deep=True)), utarg_downcast.dtype

In [ ]:
# ostatecznie uzyskamy
df2['Utarg'] =  pd.to_numeric(df1["Utarg"], downcast='float')
sizeof_fmt(sum(df2.memory_usage(deep=True)))

In [ ]:
df2.info()
df2.describe()

In [ ]:
#Porównanie czasów wykonania dla oryginalnej ramki oraz ramki zoptymalizowanej
start = datetime.now()
display(df1.groupby(['Sprzedawca']).agg({'Utarg': ['mean']}))
print(f'Czas: {datetime.now() - start}')
start = datetime.now()
display(df2.groupby(['Sprzedawca']).agg({'Utarg': ['mean']}))
print(f'Czas: {datetime.now() - start}')

### Zadanie 3
Wykonaj 3 wybrane operacje (grupowanie + agregacja, filtrowanie, itp.) na całej ramce i zmierz czas wykonania na danych oryginalnych i zoptymalizowanych. Wyświetl te czasy.

### Zadanie 4
Zapisz ramkę jako plik csv, z nagłówkami kolumn, bez indeksu. Sprawdź jaka jest różnica w wielkości pliku csv i sumy wielkości plików w formacie parquet (w eksploratorze, nie trzeba tego robić z poziomu kodu).

### Zadanie 5
Zmierz czas wczytywania danych z pliku csv dla 3 przypadków:

cały plik na raz,
cały plik ze wskazaniem parametru `chunksize` (możesz poeksperymentować z wielkością tego parametru),
z użyciem multiprocessingu zaprezentowanego w przykładzie (wcześniej podziel plik na kilka mniejszych), wskazując ilość procesów jako `ilość_rdzeni - 2` oraz drugi przypadek `(ilosc_rdzeni - 2) * 2`.

### Zadanie 6 (z gwiazdką, nie jest obowiązkowe, ale pouczające)

Wczytaj każdy plik podzielony w zadaniu 5 do oddzielnej ramki danych. Dla każdej ramki policz sumę na kolumnie likes, a następnie policz sumę tych sum. Tę część zadania wykonaj sekwencyjnie. Teraz wykorzystując multiprocessing (i przykłady z labu) wykonaj to samo zadanie zrównoleglając je. Zmierz czas obu przypadków i go wyświetl.